<h1 style="background: #FF7F50; color: #1F51FF; text-align: center; line-height: 130%; letter-spacing: 1px; font-weight: bold;">Permutations<br>with Lehmer and Comtet</h1>
<h4>Peter Luschny, July 2022</h4>

##### Outline:


<ul>
<li>Permutations

	=> select by property of associated Lehmer code
       and/or by Comtet constraint
</li>

<li>Partitions

	=> sort by A080577

<li>Partition Triangle
    
    => expand by largest part
    n >= 0, 0 <= k <= A000041(n)
</li>

<li>Reduced Triangle 
    
	=> sum by largest part
    n >= 1, 0 <= k < n
</li>

<li>Sequence
    
    => sum rows of the partition triangle
    n >= 0
</li>
</ul>


##### Concerned with:

|Partition <br> triangle|Reduced <br> triangle|Sequence|Comtet<br>constraint|Lehmer <br> constraint|
| :---:   | :---:   | :---:   | :---:  |  :---:  |
| [A355777](https://oeis.org/A355777) | [A173018](https://oeis.org/A173018) | [A000142](https://oeis.org/A000142) | none | none |
| [A355776](https://oeis.org/A355776) | [A356116](https://oeis.org/A356116) | [A056986](https://oeis.org/A56986)   | none | not<br>monotonic |
| [A356113](https://oeis.org/A356113) | [A174159](https://oeis.org/A174159) | [A356118](https://oeis.org/A356118) | none | none |
| [A134264](https://oeis.org/A134264) | [A001263](https://oeis.org/A001263) | [A000108](https://oeis.org/A000108) | none | weakly <br> decreasing |
| [A356262](https://oeis.org/A356262) | [A356263](https://oeis.org/A356263) | [A003319](https://oeis.org/A003319) | reducible | none |
| [A356264](https://oeis.org/A356264) | [A356265](https://oeis.org/A356265) | [A356291](https://oeis.org/A356291) | irreducible | none |
| [A356261](https://oeis.org/A356261) | [A119308](https://oeis.org/A119308) | [A071724](https://oeis.org/A071724) | reducible | weakly <br> decreasing |
| [A356266](https://oeis.org/A356266) | [A356115](https://oeis.org/A356115) | [A120588](https://oeis.org/A120588) | irreducible | weakly <br> decreasing |

##### Comment:

<p style="color:brown;font-size:large">By a partition triangle, we understand an irregular triangle where each row corresponds to a mapping of Partitions(n) -> ZZ. We assume a fixed order of the partitions given. Here we will use the ordering defined in A080577 (see below). Examples are A355776, A355777, and A134264. 'Reducing' then means summing the values corresponding to the partitions of n with largest part k. The 'reduced partition triangle' is a regular triangle with T(n, k) with n &ge; 1 and 0 &leq; k &lt; n.</p>

<p style="color:brown;font-size:large">Conversely, for instance, A355776, the statistic of permutations whose Lehmer code is nonmonotonic, can be seen as a refinement of A356116, which in turn is a refinement of the sequence A056986, the number of permutations on [n] containing any given pattern alpha in the symmetric group S_3.</p>

<p style="color:brown;font-size:large">So that there is no misunderstanding: The programs here attempt to trace a conceptual framework; no claim is made that they are any good for efficient computation.</p>

In [1]:
import collections
from functools import cache
from typing import Callable
from sage.all import ZZ
from sage.combinat.permutation import Permutation, Permutations
from sage.combinat.partition import Partition, Partitions, number_of_partitions

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Some utility functions used later as constrains.
</h3>

<p style="color:brown;font-size:large">A permutation p in [n] (where n >= 0) is reducible if there exist an i in 1..n-1 such that for all j in the range 1..i and all k in the range i+1..n it is true that p(j) < p(k). (Note that a range a..b includes a and b.) If such an i exists we say that i splits the permutation at i.</p>

<p style="color:brown;font-size:large">Examples: a(n) = A003319(n).</p>
<li style="color:brown;font-size:large">
    () is not reducible since there is no index i which splits (). (=> a(0) = 1)</li>
<li style="color:brown;font-size:large">
    (1) is not reducible since there is no index i which splits (1). (=> a(1) = 1)</li>
<li style="color:brown;font-size:large">
    (1, 2) is reducible since index 1 splits (1, 2) as p(1) < p(2).</li>
<li style="color:brown;font-size:large">
    (2, 1) is not reducible since at the only potential splitting point i = 1 we have p(1) > p(2). (=> a(2) = 1)</li>
<li style="color:brown;font-size:large">
For n = 3 we have (1, 2, 3), (1, 3, 2), and (2, 1, 3) are reducible and (2, 3, 1), (3, 1, 2), and (3, 2, 1) are irreducible.</li>

In [2]:
def reducible(p) -> bool:  # p is a Sage-Permutation
    return any(i for i in range(1, p.size())
                  if all(p(j) < p(k)
                      for j in range(1, i + 1)
                          for k in range(i + 1, p.size() + 1)
              )   )


def irreducible(p) -> bool:
    return not reducible(p)


def weakly_increasing(L: list[int]) -> bool:
    return all(x <= y for x, y in zip(L, L[1:]))


def weakly_decreasing(L: list[int]) -> bool:
    return all(x >= y for x, y in zip(L, L[1:]))


def not_monotonic(L: list[int]) -> bool:
    return not weakly_increasing(L) and not weakly_decreasing(L)


def void(L) -> bool:
    return True

<p style="color:brown;font-size:large">Let's illustrate this with the first ten permutations (A030298, prepended with the empty permutation).</p>


| n |permut| reduc| lehmer| notmono| decreas|
|:-| :- | :--  |     :--- | :--- | :--- |
|0|()   | False| []       | False| True|
|1|(1)  | False| [0]      | False| True|
|2|(12) | True | [0, 0]   | False| True|
|2|(21) | False| [1, 0]   | False| True|
|3|(123)| True | [0, 0, 0]| False| True|
|3|(132)| True | [0, 1, 0]| True |False|
|3|(213)| True | [1, 0, 0]| False| True|
|3|(231)| False| [1, 1, 0]| False| True|
|3|(312)| False| [2, 0, 0]| False| True|
|3|(321)| False| [2, 1, 0]| False| True|

<p style="color:brown;font-size:large">Indexing and the reduction of a partition triangle to a 'normal' triangle.</p> 

<p style="color:brown;font-size:normal">The considered sequences have three types, which are all enumerated according to the same scheme.</p>

<p style="color:brown;font-size:normal"> A 'partition triangle' starts with row n = 0 and for k the terms P(n, k) cover the range 0 <= k <= A000041(n). The partitions are sorted as in A080577, first by sum, then by largest part, and finally reverse-lexicographically.</p>

<p style="color:brown;font-size:normal"> We cite Franklin T. Adams-Watters: 
"This is the 'Mathematica' ordering of the partitions. [...] The partitions of each integer are in reverse order of the conjugates of the partitions in Abramowitz and Stegun order (A036036). They are in the reverse of the order of the partitions
in Maple order (A080576)."</p>

<p style="color:brown;font-size:normal">Generating the partitions in this order is described as 'Algorithm P' in Knuth's TAOCP, 7.2.1.4.</p>

<pre>[n\k]  0    1     2      3     4       5       6
[0]   ();
[1]   (1);
[2]   (2), (11);
[3]   (3), (21), (111);
[4]   (4), (31), (22), (211), (1111);
[5]   (5), (41), (32), (311), (221), (2111), (11111);</pre>

<p style="color:brown;font-size:normal">A 'reduced partition triangle' starts with row n = 1 and the row-index k covers the range 0 <= k < n. Terms are
T(n, k) = Sum_{&lambda;(p)=k} P(n, k), where &lambda;(p) is the largest 
part of p. We illustrate this with the brackets |...|:</p>

<pre>[n\k]  0     1         2             3              4  
[1]   (1);
[2]   (2), (11);
[3]   (3), (21),  (111);
[4]   (4), (31), |(22), (211)|,  (1111);
[5]   (5), (41), |(32), (311)|, |(221), (2111)|, (11111);</pre>

<p style="color:brown;font-size:normal">One can justifiably argue that one should also start the rows of this triangle at zero. In the OEIS this would have led to irregular triangles ('tabf'). However, we wanted to keep the same row index as in the partition triangle and get regular triangles. On the other hand one could also argue that in this case the top of the reduced triangle should be set to the index (1, 1) to get symmetry in the indices.</p>

<p style="color:brown;font-size:normal">Finally, the row sums of the partition triangle are listed. So they always start with the index n = 0.</p>


In [3]:
@cache
def Pn(n: int, k: int) -> int:
    if k == 0: return 0
    if n == 0 or k == 1: return 1
    return Pn(n, k - 1) + Pn(n - k, k) if k <= n else Pn(n, k - 1)


def reduce_parts(fun: Callable, n: int) -> list[int]:
    funn: list[int] = fun(n)
    return [sum(funn[Pn(n, k):Pn(n, k + 1)]) for k in range(n)]


def reduce_partition_triangle(fun: Callable, n: int) -> list[list[int]]:
    return [reduce_parts(fun, k) for k in range(1, n)]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Permutation statistics
</h3>

<p style="color:brown;font-size:large">The main function. Maps permutations to partitions subject to constraints on their Lehmer code.</p>

<p style="color:brown;font-size:large">The order of the integer partitions is the Abramowitz-Stegun ordering except that the finer order is reverse-lexicographic instead of lexicographic, as defined by Gus Wiseman in A334439.</p>

In [4]:
def perm_stats(n: int, lehmer_constraint: Callable):
    res = collections.defaultdict(int)
    for p in Permutations(n):
        l: list[int] = p.to_lehmer_code()
        if lehmer_constraint(l):
            c: list[int] = [l.count(i) for i in range(len(p)) if i in l]
            res[Partition(reversed(sorted(c)))] += 1
    return sorted(res.items(), key=lambda x: len(x[0]))

<p style="color:brown;font-size:large">If additionally we put a constraint on the permutations:</p>

In [5]:
def perm_red_stats(n: int, part_costraint: Callable, lehmer_constraint: Callable):
    res = collections.defaultdict(int)
    for p in Permutations(n):
        if not part_costraint(p): continue
        l: list[int] = p.to_lehmer_code()
        if lehmer_constraint(l):
            c: list[int] = [l.count(i) for i in range(len(p)) if i in l]             
            res[Partition(reversed(sorted(c)))] += 1
    return sorted(res.items(), key=lambda x: len(x[0]))

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Example of the various mappings involved, the permutations of {1, 2, 3, 4}.
</h3>

<p style="color:brown;font-size:large">The 'type' indicates whether the Lehmer code is weakly increasing, weakly decreasing or not monotonic.</p>



|Permutation|Reducible|Lehmer Code|Type |Partition|
| :---:     | :---:   | :---:     |:---:| :---    |
|1234| T | [0000]| [T,T,F]| [4]|  
|1243| T |[0010]| [F,F,T]| [3, 1]|  
|1324| T |[0100]| [F,F,T]| [3, 1]|  
|1423| T |[0200]| [F,F,T]| [3, 1]|  
|2134| T |[1000]| [F,T,F]| [3, 1]|  
|3124| T |[2000]| [F,T,F]| [3, 1]|  
|4123| F |[3000]| [F,T,F]| [3, 1]|  
|2341| F |[1110]| [F,T,F]| [3, 1]|  
|1342| T |[0110]| [F,F,T]| [2, 2]|  
|2143| T |[1010]| [F,F,T]| [2, 2]|  
|2314| T |[1100]| [F,T,F]| [2, 2]|  
|3412| F |[2200]| [F,T,F]| [2, 2]|  
|1432| T |[0210]| [F,F,T]| [2, 1, 1]| 
|2413| F |[1200]| [F,F,T]| [2, 1, 1]| 
|2431| F |[1210]| [F,F,T]| [2, 1, 1]| 
|3142| F |[2010]| [F,F,T]| [2, 1, 1]| 
|3214| T |[2100]| [F,T,F]| [2, 1, 1]| 
|3241| F |[2110]| [F,T,F]| [2, 1, 1]| 
|3421| F |[2210]| [F,T,F]| [2, 1, 1]| 
|4132| F |[3010]| [F,F,T]| [2, 1, 1]| 
|4213| F |[3100]| [F,T,F]| [2, 1, 1]| 
|4231| F |[3110]| [F,T,F]| [2, 1, 1]| 
|4312| F |[3200]| [F,T,F]| [2, 1, 1]| 
|4321| F |[3210]| [F,T,F]| [1, 1, 1, 1]| 



<p style="color:brown;font-size:large">This table can be nicely summarized in this mapping:</p>

|Partition|Counts|
| :--: | :--: |
|[4]| 1|
|[3, 1]| 7|
|[2, 2]| 4|
|[2, 1, 1]| 11|
|[1, 1, 1, 1] |1|

<p style="color:brown;font-size:large">The counts are row 4 of A355777.</p>

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A355777
</h3>


<p style="color:brown;font-size:large">A355777 -- A statistic of permutations given by their Lehmer code refining Euler's triangle A173018.</p>

In [6]:
@cache
def A355777_row(n: int) -> list[int]:
    return [v[1] for v in perm_stats(n, void)]


def A355777(n: int, k: int) -> int:
    return A355777_row(n)[k]

In [7]:
for n in range(8):
    print(A355777_row(n))

# for n in range(8):
#    print([A355777(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 4, 1]
[1, 7, 4, 11, 1]
[1, 11, 15, 32, 34, 26, 1]
[1, 16, 26, 15, 76, 192, 34, 122, 180, 57, 1]
[1, 22, 42, 56, 156, 474, 267, 294, 426, 1494, 496, 423, 768, 120, 1]


<p style="color:brown;font-size:large">Euler's A173018 -- Reduced partition triangle A355777.</p> 

In [8]:
def A173018_row(n: int) -> list[int]:
    return reduce_partition_triangle(A355777_row, n + 1)[n - 1]

def A173018(n: int, k: int) -> int:
    return A173018_row(n)[k]

In [9]:
for n in range(1, 9): 
    print(A173018_row(n))
    
#for n in range(1, 9):
#    print([A173018(n, k) for k in range(n)])

[1]
[1, 1]
[1, 4, 1]
[1, 11, 11, 1]
[1, 26, 66, 26, 1]
[1, 57, 302, 302, 57, 1]
[1, 120, 1191, 2416, 1191, 120, 1]
[1, 247, 4293, 15619, 15619, 4293, 247, 1]


<p style="color:brown;font-size:large">A000142 -- Factorial numbers, row sums of A355777.</p>

In [10]:
[sum(A355777_row(n)) for n in range(9)]

[1, 1, 2, 6, 24, 120, 720, 5040, 40320]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A355776
</h3>



<p style="color:brown;font-size:large">A355776 -- A statistic of permutations whose Lehmer code is nonmonotonic, refining triangle A356116.</p>


In [11]:
@cache
def A355776_row(n: int) -> list[int]:
    if n < 2:
        return [0]
    S = perm_stats(n, not_monotonic)
    return [0] + [s[1] for s in S] + [0]


def A355776(n: int, k: int) -> int:
    return A355776_row(n)[k] if n > 0 else 0

In [12]:
for n in range(8):
    print(A355776_row(n))

#for n in range(8):
#    print([A355776(n, k) for k in range(number_of_partitions(n))])

[0]
[0]
[0, 0]
[0, 1, 0]
[0, 3, 2, 5, 0]
[0, 6, 10, 22, 24, 16, 0]
[0, 10, 20, 12, 61, 162, 29, 102, 150, 42, 0]
[0, 15, 35, 49, 135, 432, 246, 273, 391, 1389, 461, 388, 698, 99, 0]


<p style="color:brown;font-size:large">A356116 -- Reduced partition triangle A355776.</p> 

In [13]:
def A356116_row(n):
    return reduce_partition_triangle(A355776_row, n + 1)[n - 1]

def A356116(n, k):
    return A356116_row(n)[k]

In [14]:
for n in range(1, 9):
    print(A356116_row(n))

#for n in range(1, 9):
#    print([A356116(n, k) for k in range(n)])

[0]
[0, 0]
[0, 1, 0]
[0, 5, 5, 0]
[0, 16, 46, 16, 0]
[0, 42, 252, 252, 42, 0]
[0, 99, 1086, 2241, 1086, 99, 0]
[0, 219, 4097, 15129, 15129, 4097, 219, 0]


<p style="color:brown;font-size:large">A056986 -- Number of permutations on {1,...,n} containing any given pattern alpha in the symmetric group S_3.</p>

In [15]:
def A056986(n: int) -> int:
    return sum(A355776_row(n))

print([A056986(n) for n in range(9)])

[0, 0, 0, 1, 10, 78, 588, 4611, 38890]


<p style="color:brown;font-size:large">A056986(0) = 0 is missing in the OEIS and should be prepended. In fact A056986(n) = n! - CatalanNumber(n), which is true for all n >= 0.</p>

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356113
</h3>


<p style="color:brown;font-size:large">A356113 -- Refining A174159, the Euler minus Narayana/Catalan triangle.<br> 
    T(n, k) = A355777(n, k) + A355776(n, k).</p>

In [16]:
@cache
def A356113(n: int, k: int) -> int:
    return A355777(n, k) + A355776(n, k)


def A356113_row(n: int) -> list[int]:
    return [A356113(n, k) for k in range(number_of_partitions(n))]

In [17]:
for n in range(8):
    print(A356113_row(n))

#for n in range(8):
#    print([A356113(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 5, 1]
[1, 10, 6, 16, 1]
[1, 17, 25, 54, 58, 42, 1]
[1, 26, 46, 27, 137, 354, 63, 224, 330, 99, 1]
[1, 37, 77, 105, 291, 906, 513, 567, 817, 2883, 957, 811, 1466, 219, 1]


<p style="color:brown;font-size:large">A174159 -- Reduced partition triangle A356113.</p>

In [18]:
def A174159_row(n: int) -> list[int]: 
    return reduce_partition_triangle(A356113_row, n + 1)[n - 1]


def A174159(n: int, k: int) -> int: 
    return A174159_row(n)[k]  

In [19]:
for n in range(1, 8):
    print(A174159_row(n))

#for n in range(1, 8):
#    print([A174159(n, k) for k in range(n)])

[1]
[1, 1]
[1, 5, 1]
[1, 16, 16, 1]
[1, 42, 112, 42, 1]
[1, 99, 554, 554, 99, 1]
[1, 219, 2277, 4657, 2277, 219, 1]


<p style="color:brown;font-size:large">A356118 -- Row sums of A174159.</p>

In [20]:
def A356118(n):
    return sum(A356113_row(n))

print([A356118(n) for n in range(9)])

[1, 1, 2, 7, 34, 198, 1308, 9651, 79210]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A134264
</h3>



<p style="color:brown;font-size:large">A134264 -- Noncrossing partitions. <br>
Refining A001263, the triangle of Narayana numbers.<br>
    T(n, k) = A355777(n, k) - A355776(n, k).</p>



In [21]:
@cache
def A134264(n: int, k: int) -> int:
    return A355777(n, k) - A355776(n, k)


def A134264_row(n: int) -> list[int]:
    return [A134264(n, k) for k in range(number_of_partitions(n))]

In [22]:
for n in range(8):
    print(A134264_row(n))

#for n in range(7):
#    print([A134264(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 3, 1]
[1, 4, 2, 6, 1]
[1, 5, 5, 10, 10, 10, 1]
[1, 6, 6, 3, 15, 30, 5, 20, 30, 15, 1]
[1, 7, 7, 7, 21, 42, 21, 21, 35, 105, 35, 35, 70, 21, 1]


<p style="color:brown;font-size:large">A001263 -- Triangle of Narayana numbers.</p>

In [23]:
def A001263_row(n):
    return reduce_partition_triangle(A134264_row, n + 1)[n - 1]

def A001263(n, k):
    return A001263_row(n)[k]

In [24]:
for n in range(1, 8): 
    print(A001263_row(n))
    
#for n in range(1, 8):
#    print([A001263(n, k) for k in range(n)])

[1]
[1, 1]
[1, 3, 1]
[1, 6, 6, 1]
[1, 10, 20, 10, 1]
[1, 15, 50, 50, 15, 1]
[1, 21, 105, 175, 105, 21, 1]


<p style="color:brown;font-size:large">A000108 -- Catalan numbers.</p>

In [25]:
def A000108(n):
    return sum(A134264_row(n))

print([A000108(n) for n in range(9)])

[1, 1, 2, 5, 14, 42, 132, 429, 1430]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356262
</h3>

In [26]:
@cache
def A356262_row(n: int) -> list[int]:
    if n < 2: return [1]
    return [0] + [v[1] for v in perm_red_stats(n, reducible, void)]


def A356262(n: int, k: int) -> int:
    return A356262_row(n)[k]

In [27]:
for n in range(8):
    print(A356262_row(n))

#for n in range(8):
#    print([A356262(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[0, 1]
[0, 2, 1]
[0, 2, 1, 9, 1]
[0, 2, 3, 24, 17, 24, 1]
[0, 2, 3, 3, 98, 29, 23, 156, 91, 55, 1]
[0, 2, 8, 4, 181, 43, 157, 113, 1085, 243, 418, 714, 360, 118, 1]


<p style="color:brown;font-size:large">A356263 -- Reduced partition triangle A356262.</p>

In [28]:
def A356263_row(n: int) -> list[int]:
    return reduce_partition_triangle(A356262_row, n+1)[n-1]


def A356263(n: int, k: int) -> int:
    return A356263_row(n)[k]

In [29]:
for n in range(1, 8): 
    print(A356263_row(n))
    
#for n in range(1, 8):
#    print([A356263(n, k) for k in range(n)])

[1]
[0, 1]
[0, 2, 1]
[0, 3, 9, 1]
[0, 5, 41, 24, 1]
[0, 8, 150, 247, 55, 1]
[0, 14, 494, 1746, 1074, 118, 1]


<p style="color:brown;font-size:large">A003319 -- Number of irreducible permutations.</p>

In [30]:
def A003319(n: int):
    return sum(A356262_row(n))


print([A003319(n) for n in range(9)])

[1, 1, 1, 3, 13, 71, 461, 3447, 29093]


<p style="color:brown;font-size:large">A much more efficient implementation would be for example:</p>

In [31]:
def A003319_list(size):
    R, C = [1], [1] + [0] * (size - 1)
    for n in (1..size - 1):
        for k in range(n, 0, -1):
            C[k] = C[k - 1] * k
        C[0] = -sum(C[k] for k in (1..n))
        R.append(-C[0])
    return R

print(A003319_list(10))

[1, 1, 1, 3, 13, 71, 461, 3447, 29093, 273343]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356264
</h3>

In [32]:
@cache
def A356264_row(n: int) -> list[int]:
    if n < 2: return [0]
    return [v[1] for v in perm_red_stats(n, irreducible, void)] + [0]


def A356264(n: int, k: int) -> int:
    return A356264_row(n)[k]

In [33]:
for n in range(8):
    print([n], A356264_row(n))    

#for n in range(1, 8):
#    print([A356264(n, k) for k in range(number_of_partitions(n))])

[0] [0]
[1] [0]
[2] [1, 0]
[3] [1, 2, 0]
[4] [1, 5, 3, 2, 0]
[5] [1, 9, 12, 15, 10, 2, 0]
[6] [1, 14, 23, 12, 47, 94, 11, 31, 24, 2, 0]
[7] [1, 20, 38, 48, 113, 293, 154, 137, 183, 409, 78, 63, 54, 2, 0]


<p style="color:brown;font-size:large">A356265 -- Reduced partition triangle A356264.</p>

In [34]:
def A356265_row(n: int) -> list[int]:
    return reduce_partition_triangle(A356264_row, n+1)[n-1]


def A356265(n: int, k: int) -> int:
    return A356265_row(n)[k]

In [35]:
for n in range(1, 8): 
    print(A356265_row(n))

#for n in range(1, 8):
#    print([A356265(n, k) for k in range(n)])

[0]
[1, 0]
[1, 2, 0]
[1, 8, 2, 0]
[1, 21, 25, 2, 0]
[1, 49, 152, 55, 2, 0]
[1, 106, 697, 670, 117, 2, 0]


<p style="color:brown;font-size:large">A356291 -- Number of reducible permutations.</p>

In [36]:
def A356291(n: int):
    return sum(A356264_row(n))


print([A356291(n) for n in range(9)])

[0, 0, 1, 3, 11, 49, 259, 1593, 11227]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Theorem: A bisection of Euler's triangle:
</h3>
<ul>
<li style="color:brown;font-size:large">Sequence version:
    <code>A356291(n) + A003319(n) = A000142(n)</code></li>
<li style="color:brown;font-size:large">Reduced version:
    <code>A356265(n, k) + A356263(n, k) = A173018(n, k)</code></li>
</ul>

<p style="color:brown;font-size:large">However, note that the partition version 
cannot be written as
<code>A356264(n, k) + A356262(n, k) = A355777(n, k).</code>
Let's test the functions:</p>

In [37]:
# Sequence version
def test1(n: int) -> int: 
    return A356291(n) + A003319(n) - factorial(n)

print([test1(n) for n in range(1, 9)])

[0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
# Reduced version
def test2(n: int, k: int) -> int: 
    return A356265(n, k) + A356263(n, k) - A173018(n, k)

for n in range(1, 9):
    print([test2(n, k) for k in range(n)])

[0]
[0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]


In [39]:
# Partition version
def test3(n: int, k: int) -> int: 
    return A356264(n, k) + A356262(n, k) - A355777(n, k)

for n in range(8):
    print([n], [test3(n, k) for k in range(number_of_partitions(n))])

[0] [0]
[1] [0]
[2] [0, 0]
[3] [0, 0, 0]
[4] [0, 0, 0, 0, 0]
[5] [0, 0, 0, 7, -7, 0, 0]
[6] [0, 0, 0, 0, 69, -69, 0, 65, -65, 0, 0]
[7] [0, 0, 4, -4, 138, -138, 44, -44, 842, -842, 0, 354, -354, 0, 0]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356261
</h3>

In [40]:
@cache
def A356261_row(n: int) -> list[int]:
    if n < 2: return [1]
    return [0] + [v[1] for v in perm_red_stats(n, reducible, weakly_decreasing)]


def A356261(n: int, k: int) -> int:
    return A356261_row(n)[k]

In [41]:
for n in range(8):
    print([n], A356261_row(n))

#for n in range(8):
#    print([A356261(n, k) for k in range(number_of_partitions(n))])

[0] [1]
[1] [1]
[2] [0, 1]
[3] [0, 2, 1]
[4] [0, 2, 1, 5, 1]
[5] [0, 2, 2, 7, 7, 9, 1]
[6] [0, 2, 2, 1, 9, 18, 3, 16, 24, 14, 1]
[7] [0, 2, 2, 2, 11, 22, 11, 11, 25, 75, 25, 30, 60, 20, 1]


<p style="color:brown;font-size:large">A119308 -- Reduced partition triangle of A, the first differences of the Catalan numbers.</p>

In [42]:
def A119308_row(n: int) -> list[int]:
    return reduce_partition_triangle(A356261_row, n + 1)[n - 1]


def A119308(n: int, k: int) -> int:
    return A119308_row(n)[k]

In [43]:
for n in range(1, 8):
    print([n], A119308_row(n))

#for n in range(8):
#    print([A119308(n, k) for k in range(n)])

[1] [1]
[2] [0, 1]
[3] [0, 2, 1]
[4] [0, 3, 5, 1]
[5] [0, 4, 14, 9, 1]
[6] [0, 5, 30, 40, 14, 1]
[7] [0, 6, 55, 125, 90, 20, 1]


$$ \text{A119308}(n, k) = \sum_{j=0}^{n-1} \frac{\binom{n-2}{j-1} \binom{j}{k} \binom{k}{j-k}}{j-k+1} $$


<p style="color:brown;font-size:large">Our triangle (and the formula) has a different offset than the one in the OEIS and an additional column on the left side.</p>

<p style="color:brown;font-size:large">A071724 (shifted) -- number of standard tableaux. </p>

In [44]:
def A071724(n: int) -> int:
    return sum(A356261_row(n))

print([A071724(n) for n in range(9)])

[1, 1, 1, 3, 9, 28, 90, 297, 1001]


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356266
</h3>

In [45]:
@cache
def A356266_row(n: int) -> list[int]:
    if n < 2: return [1]
    return [0] + [v[1] for v in perm_red_stats(n, irreducible, weakly_decreasing)]


def A356266(n: int, k: int) -> int:
    return A356266_row(n)[k]

In [46]:
for n in range(8):
    print(A356266_row(n))

#for n in range(8):
#    print([A356266(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[0, 1]
[0, 1, 1]
[0, 1, 2, 1, 1]
[0, 1, 3, 3, 3, 3, 1]
[0, 1, 4, 4, 2, 6, 12, 2, 4, 6, 1]
[0, 1, 5, 5, 5, 10, 20, 10, 10, 10, 30, 10, 5, 10, 1]


<p style="color:brown;font-size:large">A356115 -- Reduced triangle of the partition triangle A356266.</p>

In [47]:
def A356115_row(n: int) -> list[int]:
    return reduce_partition_triangle(A356266_row, n + 1)[n - 1]


def A356115(n: int, k: int) -> int:
    return A356115_row(n)[k]

In [48]:
for n in range(1, 8):
    print([n], A356115_row(n))

#for n in range(8):
#    print([A356115(n, k) for k in range(n)])

[1] [1]
[2] [0, 1]
[3] [0, 1, 1]
[4] [0, 3, 1, 1]
[5] [0, 4, 6, 3, 1]
[6] [0, 9, 20, 6, 6, 1]
[7] [0, 11, 45, 50, 15, 10, 1]


<p style="color:brown;font-size:large">A120588 -- the shifted Catalan numbers.</p>

In [49]:
def A120588(n: int) -> int:
    return sum(A356266_row(n))

print([A120588(n) for n in range(8)])

[1, 1, 1, 2, 5, 14, 42, 132]


<p style="color:brown;font-size:large">~~~</p>